# FIBROBLAST

This chapter describes the implementation of the Fibroblast agent

# Fibroblast
Class representing fibroblast cells in the model.
Fibroblast cells can induce endothelial growth, migrate, proliferate, and interact with endothelial cells.

Attributes:
    unique_id: The unique identifier for the fibroblast agent.
    position: The position of the agent in the model grid.
    model: The model in which the agent resides.
    alive: Indicates whether the fibroblast agent is alive.
    proliferation_capacity: The remaining capacity for fibroblast proliferation.
    prob_support_growth: The probability of supporting tumor cell growth.
    Tumor Cells Class Attributes:
    unique_id: The unique identifier for the tumor cell agent.
    position: The position of the agent in the model grid.
    model: The model in which the agent resides.
    viable: Indicates if the tumor cell is viable.
    proliferation_prob: Probability of tumor cell proliferation.
    death_prob: Probability of tumor cell death.
    nearest_dist: The distance to the nearest endothelial cell.
    hypoxia_thresholds: Thresholds for different levels of hypoxia.
    prolif_inhib_intensity: Intensity of proliferation inhibition.
    death_intensity: Intensity of cell death.

In [ ]:
## Imports and Class Declaration
Import the necessary prerequisites for this class to run as intended.

**Class Declaration**
class Fibroblast(Agent): 

In [ ]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from Tumor_cells import Tumor_cells
# from mesa.visualization.modules import CanvasGrid
# from mesa.visualization.ModularVisualization import ModularServer

## Instance Initiation
Instance initalization associates the instance with different attribute parameter values depending on initialization conditions. 


**Declaration**


def __init__(self, unique_id, position, model, *args):


Initialize a fibroblast cell with the given parameters.


Args:
    unique_id: Unique identifier for the agent.
    position: The position of the agent on the grid as a tuple (x, y).
    model: The model to which the agent belongs.
    *args: Optional arguments, including the previous distance to the nearest endothelial cell.


**Attributes**


There are 8 groups of instance attributes and parameters that relate to different components of fibroblast agents. These are: (1) constructor initiation attributes, (2) basic parameter values, (3) kinetic parameters, (4) hypoxia parameters, (5) parameters related to endothelial cell tracking, (6) fibroblast-tumor cell interaction parameters, (7) age-related parameters, and (8) hunger-related parameters.



In [ ]:

# Fibroblast Agent
"""
    Represents a fibroblast agent in the model.

    Attributes:
        position (tuple): The (x, y) position of the agent in the grid.
        alive (bool): Indicates whether the agent is alive.
        proliferation_capacity (int): The remaining capacity for the agent to proliferate.
"""
    #PARAMETERS
# Define parameters for the Fibroblast agent
params = {
    "Fpdeath": 0.0018*10,  # Probability of death 
    "Fpmig": 1.4,          # Probability of migration
    "Fpprol": 0.0838/4,    # Probability of proliferation
    "Fpmax": 4             # Initial proliferation capacity
}

class Fibroblast(Agent):
    """
        Initializes a Fibroblast agent.

        Args:
            unique_id (int): Unique identifier for the agent.
            position (tuple): Initial position of the agent in the grid (x, y).
            model (Model): The model the agent belongs to.
    """
       #CONSTRUCTOR
    def __init__(self, unique_id, position, model):
        super().__init__(unique_id, model)
        self.position = position

        #BASICS
        self.alive = True
        self.proliferation_capacity = params["Fpmax"]

        #INTERACTION PARAMETERS
        self.prob_support_growth = 0.05
    

## Eat
Consumes a specified amount of nutrition.
Allows the fibroblast agent to interact with the environment by consuming resources. This action could impact its behavior or future actions.
Nutrition is likely tied to the agent's ability to perform tasks like proliferation, migration, or tumor support.

In [ ]:
def eat(self, val):
        self.model.eat_nutrition(val)
    
    """
        Executes one step for the fibroblast agent, including:
            - Death: Agent may die based on the `Fpdeath` probability.
            - Migration: Agent may move to a neighboring cell based on the `Fpmig` probability.
            - Proliferation: Agent may create a new fibroblast in an adjacent cell if it has 
              proliferation capacity and the `Fpprol` probability is met.
    """


## Step
Executes one step of the fibroblast agent's lifecycle.

### Purpose:
This method defines the agent's behavior during each step of the simulation, including:
- **Migration:** Moving to a neighboring cell based on a probability.
- **Tumor Support:** Supporting nearby tumor cells.
- **Proliferation:** Creating a new fibroblast under certain conditions.
- **Death:** Determining if the agent dies based on a probability.
This is the main driver of the agent's actions, simulating realistic fibroblast behaviors like migration, interaction with tumors, and eventual death.


In [ ]:

    # Step Logic
    def step(self):
        self.eat(3)
    
        # Migration
        if self.random.random() < params["Fpmig"]:
            self.migrate()

        # Support tumor growth
        if self.random.random() < self.prob_support_growth:
            self.support_tumor_cells()

        #Proliferation
        #Elif self.proliferation_capacity > 0 and self.random.random() < params["Fpprol"]:
        #self.proliferate()

        # Death
        if self.random.random() < params["Fpdeath"]:
            self.alive = False
            self.model.grid.remove_agent(self)
            self.model.schedule.remove(self)
            return


    """
        Moves the agent to a random neighboring cell if possible.
    """



## Migration
Moves the fibroblast agent to a random neighboring cell, if possible.
Simulates the migration behavior of fibroblasts, allowing them to explore the environment and interact with other agents.
Migration is crucial for fibroblast roles in processes like wound healing and tumor progression, where they need to move to specific locations.


In [ ]:

    # Migration Logic
    def migrate(self):
        possible_steps = self.model.grid.get_neighborhood(self.position, moore=True, include_center=False)
        
        # Filter only empty positions
        empty_positions = [pos for pos in possible_steps if self.model.grid.is_cell_empty(pos)]

        # Pick an empty position if there are any
        if len(empty_positions) > 0:
            new_position = self.random.choice(empty_positions)
            if new_position != None:
                self.model.grid.move_agent(self, new_position)


## Proliferation
Creates a new fibroblast agent in an empty neighboring cell if one exists. Decreases the proliferation capacity of the current agent.
Simulates the fibroblast's ability to replicate under favorable conditions, contributing to population growth.
Proliferation is a key feature of fibroblast behavior, particularly in tumor microenvironments or tissue repair scenarios. However, it will not function during these tests to keep a specific ratio of tumor to fibroblast cells


In [ ]:

    """
        Creates a new fibroblast agent in an empty neighboring cell if one exists.
        Reduces the proliferation capacity of the current agent by 1.
    """

    def proliferate(self):
        self.model.generate_agents(Fibroblast, "proliferate", 1, self.position)
        empty_cells = [cell for cell in self.model.grid.get_neighborhood(self.position, moore=True, include_center=False)
                       if self.model.grid.is_cell_empty(cell)]
        #If empty_cells:
            #new_position = self.random.choice(empty_cells)
            #new_agent = Fibroblast(self.model.next_id(), new_position, self.model)
            #self.model.grid.place_agent(new_agent, new_position)
            #self.model.schedule.add(new_agent)
            #self.proliferation_capacity -= 1


## Tumor support growth
Enhances the growth and survival of nearby tumor cells.
Increases tumor cell proliferation by modifying their proliferation inhibition and death intensity based on proximity and environmental conditions.
Represents the supportive role fibroblasts can play in tumor progression, as they often interact with tumor cells to promote their growth and survival.

In [ ]:

    # Support Tumor Cells Logic
    def support_tumor_cells(self):
        neighbors = self.model.grid.get_neighbors(self.position, moore=True, include_center=False)
        for neighbor in neighbors:
            if isinstance(neighbor, Tumor_cells):
            
            # Support tumor growth with probability
                if self.random.random() < self.prob_support_growth:
            
                # Create a new tumor cell in a random neighboring position
                    neighbors = self.model.grid.get_neighborhood(neighbor.position, moore=True, include_center=False)
                    tumor_cells = [cell for cell in neighbors if isinstance(cell, Tumor_cells)]
            
                    if tumor_cells:
                        tumor_cell = self.random.choice(tumor_cells)
                        if tumor_cell.nearest_dist > tumor_cell.hypoxia_thresholds[1]:
                            tumor_cell.prolif_inhib_intensity = 0.3
                            tumor_cell.death_intensity = 0.5
                            print("Fibroblast Supperoted TUMOR PROLIFERATION")